In [1]:
# Проверка работоспособности 2-х моделей  разбиением пользователей
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from datetime import datetime
from sqlalchemy import create_engine
from loguru import logger

In [19]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [11]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [20]:
# Для get_exp_group нужны только users
user_data = pd.read_sql('SELECT * FROM public.user_data', con = engine)

In [105]:
import hashlib

def get_exp_group(user_id: int) -> str:
    salt = 'my_salt'
    group_count = 2
    value_str = str(user_id) + salt
    value_num = int(hashlib.md5(value_str.encode()).hexdigest(), 16)
    if value_num % group_count == 0:
        return 'test'
    else:
        return 'control'

#group = get_exp_group(80000)
#print(f"User 200 в группе: {group}")

User 200 в группе: test


In [12]:
def load_features() -> pd.DataFrame:
    feed_data = batch_load_sql("""
        SELECT *
        FROM public.feed_data
        WHERE action='like'
        limit 9000000""")
    return feed_data

preprocess_posts_new = pd.read_sql('SELECT * FROM public.kazakov_mta3669_features_dl', con = engine)
posts = pd.read_sql('SELECT * FROM public.post_text_df', con = engine)
posts.set_index('post_id', inplace=True) 


In [80]:
def get_model_path(path: str, exp_group: str) -> str:
    """Полный путь к модели с учетом среды и группы"""
    if os.environ.get("IS_LMS") == "1":
        base_dir = '/workdir/user_input/'
        model_dir = 'model_test' if exp_group == 'test' else 'model_control'
        return os.path.join(base_dir, model_dir, os.path.basename(path))
    return path
    

In [35]:
#features_data = load_features()
print(features_data['timestamp'].min())
print(features_data['timestamp'].max())
print(user_data['user_id'].min())
print(user_data['user_id'].max())
print(user_data['user_id'].shape[])

2021-10-01 06:09:21
2021-12-29 23:46:10
200
168552
()


In [111]:
request = {
    'id': 160000,
    'time': datetime(2023, 5, 17),
    'limit': 3
}

In [81]:
def load_model(exp_group: str, model_filename: str) -> CatBoostClassifier:
    """Загрузка модели для указанной группы"""
    model_path = get_model_path(model_filename, exp_group)
    model = CatBoostClassifier()
    model.load_model(model_path)
    return model

In [86]:
import os

model_control = load_model('control', 'catboost.cbm')
model_test = load_model('test', 'catboost_model_mta3669.cbm')

In [112]:
#Функция классического ML
def recommendations_ML(id: int, time: datetime, limit: int):
    user_view_posts = features_data[features_data['user_id'] == id]['post_id'].unique() 
    select_posts = preprocess_posts[~preprocess_posts['post_id'].isin(user_view_posts)]
    user_init = user_data[user_data['user_id'] == id]
    select_posts_use = select_posts.copy() 
    for col in user_init.columns:
        select_posts_use = select_posts_use.assign(**{col: user_init[col].iloc[0]})
    select_posts_use = select_posts_use.set_index(['user_id', 'post_id'])
    viewing_posts = features_data[features_data['user_id'] == id]['target'].count()
    like_posts = features_data[(features_data['user_id'] == id) & (features_data['target'] == 1)]['target'].count()
    like_frequency = like_posts / viewing_posts if viewing_posts > 0 else 0
    user_dates = features_data[features_data['user_id'] == id]['timestamp']
    mean_date = user_dates.mean()
    min_date = user_dates.min()
    select_posts_use['target'] = like_frequency
    select_posts_use['min_date'] = min_date
    categorical_features = ['country', 'city', 'os', 'source']
    C = 0.006
    for col in categorical_features:
        if col in ['os', 'source']:
            if select_posts_use[col].nunique() == 2: 
                dummies = pd.get_dummies(select_posts_use[col], prefix=col, dtype=int, drop_first=True)
                select_posts_use = pd.concat([select_posts_use, dummies], axis=1)
                select_posts_use = select_posts_use.drop(col, axis=1)
            else:
                if col == 'os':
                    select_posts_use['os_iOS'] = (select_posts_use[col] == 'iOS').astype(int)
                elif col == 'source':
                    select_posts_use['source_organic'] = (select_posts_use[col] == 'organic').astype(int)
                select_posts_use = select_posts_use.drop(col, axis=1)
        else:
            means = select_posts_use.groupby(col)['target'].mean()
            noisy_means = means + C * np.random.randn(len(means))
            mapping_dict = noisy_means.to_dict()
            select_posts_use[col] = select_posts_use[col].map(mapping_dict)
    
    select_posts_use = select_posts_use.drop(['target'], axis = 1) 
    select_posts_use['day'] = select_posts_use['min_date'].dt.dayofweek
    select_posts_use['month'] = select_posts_use['min_date'].dt.month
    select_posts_use['hour'] = select_posts_use['min_date'].dt.hour
    select_posts_use = select_posts_use.drop(['min_date'],axis=1)
    
    #expected_features = model_control.feature_names_
    #print("Ожидаемые признаки:", expected_features)
    #print("Фактические признаки:", select_posts_use.columns.tolist())
    
    select_posts_use['predict'] = model_control.predict_proba(select_posts_use)[:,1]
    typle_idx = select_posts_use['predict'].nlargest(limit).index.tolist()
    posts_idx = []
    for item in typle_idx:
        posts_idx.append(item[1])
    #Вывести мы должны список из словарей
    final_posts = []
    for idx in posts_idx:
        myDict = {
            'id':  idx,
            'text': posts.loc[idx, 'text'],
            'topic': posts.loc[idx, 'topic'],
        }
        final_posts.append(myDict)
    #accord = {'exp_group': 'control'}
    #final_posts.append(accord)
    return final_posts
      

In [109]:
#Функция DL
def recommendations_DL(id: int, time: datetime, limit: int):
    # Сначала удалим посты из выборки, которые пользователь уже видел
    user_view_posts = (features_data[features_data['user_id'] == id]
    [['post_id', 'timestamp']].drop_duplicates(subset=['post_id']).reset_index(drop=True)
                      )
    viewed_ids = user_view_posts['post_id'].tolist() 
    select_posts = preprocess_posts_new[~preprocess_posts_new['post_id'].isin(viewed_ids)]
    #return select_posts
    # определим фичи конкретного юзера
    user_init = user_data[user_data['user_id'] == id]
    # Добавляем их к табл. select_posts
    select_posts_use = select_posts.copy()
    user_dates = features_data[features_data['user_id'] == id]['timestamp']
    user_dates_mean = user_dates.mean()
    select_posts_use.insert(0, 'month', user_dates_mean.month)
    select_posts_use.insert(1, 'hour', user_dates_mean.hour)
    for col in user_init.columns:
        select_posts_use = select_posts_use.assign(**{col: user_init[col].iloc[0]})
    new_order = list(user_init.columns) + [col for col in select_posts_use.columns if col not in user_init.columns]
    select_posts_use = select_posts_use[new_order]
    # Создаем мультииндекс user_id / post_id
    select_posts_use = select_posts_use.set_index(['user_id', 'post_id'])
    select_posts_use.drop(['index'], axis = 1, inplace=True)
    select_posts_use['predict'] = model_test.predict_proba(select_posts_use)[:,1]
    #return select_posts_use
    typle_idx = select_posts_use['predict'].nlargest(limit).index.tolist()
    posts_idx = []
    for item in typle_idx:
        posts_idx.append(item[1])
    #Вывести должны мы список из словарей
    final_posts = []
    for idx in posts_idx:
        myDict = {
            'id':  idx,
            'text': posts.loc[idx, 'text'],
            'topic': posts.loc[idx, 'topic'],
        }
        final_posts.append(myDict)
    #accord = {'exp_group': 'test'}
    #final_posts.append(accord)
    return final_posts

In [117]:
exp_group = get_exp_group(request['id'])
if exp_group == 'test':
    logger.info(f'Dl model for user {request['id']}')
    output_function = recommendations_DL(request['id'], request['time'], request['limit'])
    output_function.append({'exp_group': 'test'})
elif exp_group == 'control':
    logger.info(f'ML model for user {request['id']}')
    output_function = recommendations_ML(request['id'], request['time'], request['limit'])
    output_function.append({'exp_group': 'control'})
else:
  raise ValueError('unknown group')
    
output_function

2025-12-06 16:37:31.965 | INFO     | __main__:<module>:7 - ML model for user 160000


[{'id': 332,
  'text': 'LSE sets date for takeover deal\n\nThe London Stock Exchange (LSE) is planning to announce a preferred takeover by the end of the month, newspaper reports claim.\n\nThe Sunday Telegraph said the LSEs plan was further evidence it wants to retain tight control over its destiny. Both Deutsche Boerse and rival Euronext held talks with the London market last week over a possible offer. A £1.3bn offer from Deutsche Boerse has already been rejected, while Euronext has said it will make an all cash bid. Speculation suggests that Paris-based Euronext has the facilities in place to make a bid of £1.4bn, while its German rival may up its bid to the £1.5bn mark. Neither has yet tabled a formal bid, but the LSE is expected to hold further talks with the two parties later this week. However, the Sunday Telegraph report added that there are signs that Deutsche Boerse chief executive Werner Seifert is becoming increasingly impatient with the LSEs managed bid process.\n\nDespite

In [95]:
preprocess_posts

,post_id,Компонента 1,Компонента 2,Компонента 3,Компонента 4,Компонента 5,Компонента 6,Компонента 7,Компонента 8,Компонента 9,...,Компонента 97,Компонента 98,Компонента 99,Компонента 100,covid,entertainment,movie,politics,sport,tech
0,1,0.116060,-0.186410,-0.049454,-0.000525,-0.101009,-0.133808,-0.202089,-0.065688,-0.073895,...,-0.010917,-0.025841,0.028009,-0.051560,0,0,0,0,0,0
1,2,0.144863,-0.213865,-0.057816,-0.049957,0.040709,-0.041332,-0.027845,0.008899,-0.021784,...,-0.023653,-0.093682,-0.024064,-0.051820,0,0,0,0,0,0
2,3,0.124525,-0.147405,-0.033435,0.010892,-0.114626,-0.132842,-0.157777,-0.064754,-0.007903,...,0.009781,0.013621,0.048129,-0.032911,0,0,0,0,0,0
3,4,0.099042,-0.150082,-0.023253,-0.003138,-0.075870,-0.127194,-0.110220,-0.029244,0.039262,...,-0.042204,-0.018020,0.000902,0.000216,0,0,0,0,0,0
4,5,0.064908,-0.077954,-0.022646,0.000431,-0.032227,-0.038098,-0.007134,0.005762,0.010590,...,-0.032575,0.000287,0.034106,-0.026841,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,0.254660,0.124200,0.017228,-0.143192,-0.034008,0.014999,0.008834,-0.018330,0.056926,...,0.035077,-0.008423,0.033310,0.016169,0,0,1,0,0,0
7019,7316,0.209761,0.108771,0.014662,-0.092212,-0.014201,-0.046052,-0.001107,0.022673,-0.030220,...,0.002406,0.023937,-0.048789,-0.027441,0,0,1,0,0,0
7020,7317,0.138492,0.056343,0.003408,0.115936,0.076048,-0.027877,0.013407,-0.041453,0.004192,...,-0.013270,-0.021687,-0.027060,-0.012615,0,0,1,0,0,0
7021,7318,0.200977,0.030445,-0.010369,0.093321,0.052157,0.010094,-0.014525,0.010085,0.099764,...,0.023916,0.000117,-0.005310,0.009476,0,0,1,0,0,0


In [78]:
exp_group = get_exp_group(request['id'])
exp_group

'control'

In [94]:
preprocess_posts = pd.read_sql('SELECT * FROM public.preprocess_posts', con = engine)

In [114]:

def test_distribution(sample_size=100000):
    """Тестирует равномерность распределения."""
    groups = {'test': 0, 'control': 0}
    
    for user_id in range(sample_size):
        group = get_exp_group(user_id)
        groups[group] += 1
    
    test_percent = (groups['test'] / sample_size) * 100
    control_percent = (groups['control'] / sample_size) * 100
    
    print(f"Test group: {test_percent:.2f}%")
    print(f"Control group: {control_percent:.2f}%")
    print(f"Difference: {abs(test_percent - control_percent):.2f}%")
    
    # Проверяем, что разница меньше 1%
    assert abs(test_percent - 50) < 1, "Распределение не 50/50"
    
    return groups

# Запуск теста
test_distribution()

Test group: 50.10%
Control group: 49.90%
Difference: 0.20%


{'test': 50098, 'control': 49902}